# Sklearn Pipelines Exercise
*Made by viga@itu.dk and thso@itu.dk*

## Introduction

In this exercise you'll be working with the [Wine Quality Dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) from the UCI Machine Learning Repository. The dataset consists of 11 features and a quality score for 4898 white wine samples and 1599 red wine samples. The goal is to predict the quality of the wine based on the features.

The datasets are located in the `data` folder. The `winequality-red.csv` file contains the red wine samples and the `winequality-white.csv` file contains the white wine samples. Lastly, the `winequality.names` file contains a description of the dataset.

The goal of this exercise is to get you familiar with the [Scikit-learn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) API. You'll be using pipelines to perform feature scaling and feature selection.

## Load in the data

You can either load the red-wine dataset or the white-wine dataset. You can also load both datasets and combine them if you want.

Both datasets are available in the `data` folder, and are called `winequality-red.csv` and `winequality-white.csv`.

Hint: You can use the `pd.read_csv()` function to load in the data (remember to check the delimiter!). You can find the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).


In [6]:
import pandas as pd


df = pd.read_csv("data/winequality-red.csv", delimiter=";")
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Data Exploration

### Check the number of missing values in the dataset.

Hint: `.isnull()`

Dont worry if there are missing values, we'll handle them later in our pipeline!

In [11]:
df.isnull()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,False,False,False,False,False,False,False,False,False,False,False,False
1595,False,False,False,False,False,False,False,False,False,False,False,False
1596,False,False,False,False,False,False,False,False,False,False,False,False
1597,False,False,False,False,False,False,False,False,False,False,False,False


### Check some basic statistics

We want to know the mean, standard deviation, minimum, maximum and quartiles of each feature.
This will give us a good idea of the distribution of the data, and also tell us if we need to do any scaling.

Hint: `.describe()`, If the output is hard to read, you can use `.T` to transpose the dataframe, i.e., swapping the rows and columns.

Do you notice anything strange about the data? Is there anything that stands out to you?

In [12]:
# check some basic statistics
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1597.000000,1596.000000,1599.000000,1598.000000,1596.000000,1597.000000,1597.000000,1597.000000,1596.000000,1596.000000,1599.000000
mean,8.319637,0.527583,0.270495,2.538806,0.087466,15.879073,46.405761,0.996748,3.310971,0.658164,10.422650,5.636023
std,1.741096,0.178980,0.194613,1.409928,0.047080,10.469052,32.817550,0.001886,0.154418,0.169647,1.065613,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997830,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


We saw that there were some missing values in the dataset, this we can fix in the pipeline, using the [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) from sklearn.

Next we also saw that there was a some differences in the scale of the different variables, so we will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from sklearn to scale the data. This will make it easier for the model to learn the patterns in the data. Especially for the KNN algorithm (which we'll use), which is based on distance, it is important that the data is scaled.

If you think of other transformations that might be useful for this dataset, feel free to try them out!

**Take a look at the [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) module for some inspiration.**

# Splitting the data

Now that we have created our pipeline, we can train the model.

First we need to split the data into a training set and a test set. We will use the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from sklearn to do this. But first we need to split the data into features and labels.

The features are all the columns in the dataset, except for the `quality` column, which are the labels.

We will use the default split of 75% training data, and 25% test data.

Hint: You can use the `random_state` parameter to make sure that the data is split the same way every time you run the code.

The train_test_split function returns four values, the first two are the training and test data, and the last two are the train and test labels.

In [21]:
from sklearn.model_selection import train_test_split
# split the data into X and y
X = df.drop(columns=["quality"])
y = df["quality"]
# now split the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Creating the pipeline

We will now create a pipeline that will handle the missing values and scaling for us, and finally train a KNN model on the data.

We will use the [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class from sklearn to create our pipeline.

The pipeline will consist of three steps, the first step will be to impute the missing values, and the second step will be to scale the data, and the third step will be to train the model.

The pipeline format is a list of tuples, where the first element in the tuple is the name of the step, and the second element is the step itself, e.g.:

```python
pipeline = Pipeline([
	('step_name', step()),
	('step_name', step()),
	('step_name', step()),
])
```

Where the `step_name` is a string, and the `step` is a sklearn object - this can be a "Transformer" object (like `SimpleImputer` and `StandardScaler`) or an "Estimator" object (like `KNeighborsClassifier` or `LinearRegression`).

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# create your pipeline
pipe = Pipeline([
    ('impute', SimpleImputer()),
    ('scale', StandardScaler()),
    ('estimator', KNeighborsClassifier()),
])

# Evaluating the model

Now that we have trained the model, we want to evaluate it to see how well it performs.

We will use the [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) function from sklearn to calculate the accuracy of the model.

Since we have created a pipeline, we can simply call the `.fit()` and `.predict()` methods on the pipeline object, and it will handle the preprocessing for us - and importantly in the correct order.

Remember to only call `.fit()` on the training data. Calling `.fit()` on the test data will cause the model to overfit to the test data, and will give you an overly optimistic accuracy score.

* **`.fit(X_train, y_train)` will train the model on the training data.**
* **`.predict(X_train)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_train).**
* **`.predict(X_test)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_test).**

In [23]:
from sklearn.metrics import accuracy_score

# fit the pipeline

pipe.fit(X_train, y_train)
print(f"train acc:{accuracy_score(y_train, pipe.predict(X_train))}")

# evaluate the pipeline

print(f"test acc: {accuracy_score(y_test, pipe.predict(X_test))}")



train acc:0.7055879899916597
test acc: 0.5825
